In [ ]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import plotly.express as px
from pandas import DataFrame as df
from EC_data_processing_lib import get_header_length
from EC_data_processing_lib import get_decimal_separator

currentcorrect = 1000 #converts to mA to A in the log
areacorrect = 1 #converts the surface area in the log
cal = 0.309 #calibration of the Hg/HgO electrode
fig = go.Figure()

CoNx_dict = {
'Ru' : 0.85,  #measured resistance in ohms
'data' : 'Data/2022_01_05 CoNx Ti electrode1_04_CP_C05.mpt', #file path the slash needs to be '/' rather than the '\' which is automatic
'label' : 'CoN<sub>X</sub>',
}

NiNx_dict = {
'Ru' : 0.85,  #measured resistance in ohms
'data' : 'Data/2022_01_04 NiNx Ti electrode1_04_CP_C05.mpt',
'label' : 'NiN<sub>X</sub>',
}

FeNx_dict = {
'Ru' : 0.85,  #measured resistance in ohms
'data' : 'Data/2022_01_11 FeNx Ti electrode set1_04_CP_C05.mpt',
'label' : 'FeN<sub>X</sub>',
}

dataset = [CoNx_dict, NiNx_dict, FeNx_dict]

for i in dataset:
    headerlength = get_header_length(i['data'])
    data = pd.read_csv(i['data'], encoding='ansi', sep="\t", decimal=get_decimal_separator(i['data']),  skiprows=range(headerlength))
    exp = data['Ns']
    Nexp = exp[-1:] + 1
    voltage = data['<Ewe>/V']
    current = data['control/mA']
    voltage_corrected = data['<Ewe>/V'] - data['control/mA'] * i['Ru']/1000 - cal #does the iR correction converts to overpotential
    j = 0
    Tafel = []
    for j in range(int(Nexp)):
        v = df.mean(voltage_corrected.loc[exp==j])
        verr = df.std(voltage_corrected.loc[exp==j])
        J = current / currentcorrect / areacorrect
        logJ = np.log10(J.loc[exp==j])
        logJ = logJ.iloc[0:1] 
        logJ = float(logJ)
        logJ = np.round(logJ,2)
        Tafel.append([v, verr, logJ]) 
    Tafel = np.array(Tafel)


    fig.add_trace(go.Scatter(
        x=Tafel[:,2],
        y=Tafel[:,0],
        error_y=dict(
            type='data',
            #color = colors,
            array=Tafel[:,1],
            visible=True),
        name= i['label']
        ))
fig.update_xaxes(range=[-6, -1.0],
    nticks=10,
    ticks='outside',
    showgrid=False, gridwidth=1, gridcolor='grey',
    zeroline=False, zerolinecolor='grey', zerolinewidth= 1,
    showline=True, linewidth=2, linecolor='black')

fig.update_yaxes(range=[0, 1],
    nticks=10,
    showgrid=False, gridwidth=1, gridcolor='grey',
    ticks='outside',
    zeroline=True, zerolinecolor='grey', zerolinewidth= 1,
    showline=True, linewidth=2, linecolor='black')
fig.update_layout(
    xaxis_title="Overpotential, η(V) - iR<sub>u</sub>",
    yaxis_title="Current (mA/cm<sup>2</sup>)",
    font=dict(
        family="Times New Roman, monospace",
        size=18,
        color="dimgrey"
        
    )
)
fig.update_traces(mode='markers')
fig.update_layout({
    "plot_bgcolor": "rgba(0, 0, 0, 0)",
    'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})
fig.update_layout(
    width = 500,
    height = 500)
fig.show()
fig.write_image('Tafel.pdf')